Import modules

In [ ]:
import importlib
from System.paths import *
from OMM_interface.openmm import *
import os
import shutil
from VMD import VMD
import importlib
import os
import numpy as np
from pathlib import Path
import re
from ASE_interface.ase_calculation import CP2K
from Coord_Toolz.mdanalysis import MDA_reader
from Direct_cp2k_calculation.direct_cp2k import Direct_Calculator
from System.paths import Paths
from System.system import Molecular_system
from ASE_interface.ase_calculation import ASE_system
from Parametrization.parametrization import Parametrization
from Parametrization.optimizers import Optimizer
from System.system import write_single_pdb
from OMM_interface.openmm import OpenMM_system

Configure Paths

In [ ]:
paths = Paths()
paths.mm_top_file = 'smallbox.psf'
paths.mm_top_file_path = '/home/ac127777/Documents/methylphosphate/cp2k_geom_opts/mp0mp0/frame0/'
paths.crd_pdb_file = 'smallbox.pdb'
paths.crd_pdb_file_path = paths.mm_top_file_path
paths.str_file = 'mp0.str'
paths.str_file_path = '../methylphosphate/force_matching/test/'
paths.traj_file = 'optcoords.xyz'
paths.traj_file_path = '/home/ac127777/Documents/methylphosphate/cp2k_geom_opts/mp0mp0/'
paths.working_dir = '../methylphosphate/force_matching/test/'
paths.project_name = 'mp0mp0'
paths.set()

Set up System.system.Molecular_system

In [ ]:
Molsys = Molecular_system('2_solvent', 'net_properties', 'forces')

Set up Coord_Toolz.mdanalysis.MDA_reader objects